In [28]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

# Parameters
NUM_PARTICLES = 30
NUM_ITERATIONS = 100
DIMENSIONS = 2
TARGET_ARRAY = np.array([1, -1])
SEARCH_SPACE = [-5, 5]

# PSO parameters
INERTIA_WEIGHT = 0.7
COGNITIVE_WEIGHT = 1.5
SOCIAL_WEIGHT = 1.5

# Set up the figure
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlim(SEARCH_SPACE[0], SEARCH_SPACE[1])
ax.set_ylim(SEARCH_SPACE[0], SEARCH_SPACE[1])
ax.set_xlabel('X Position')
ax.set_ylabel('Y Position')
ax.set_title('Particle Swarm Optimization Convergence')
ax.grid(True)

# Target marker
target_marker = ax.scatter(TARGET_ARRAY[0], TARGET_ARRAY[1], c='red', s=200, marker='*', label='Target')
particles_scatter = ax.scatter([], [], c='blue', alpha=0.5, label='Particles')
global_best_scatter = ax.scatter([], [], c='green', s=100, marker='X', label='Global Best')
ax.legend()

# Objective function
def objective_function(x):
    return np.sum((x - TARGET_ARRAY) ** 2)

class PSO_Visualizer:
    def __init__(self):
        self.particles = np.random.uniform(low=SEARCH_SPACE[0], high=SEARCH_SPACE[1],
                                         size=(NUM_PARTICLES, DIMENSIONS))
        self.velocities = np.random.uniform(low=-1, high=1, size=(NUM_PARTICLES, DIMENSIONS))
        self.personal_bests = self.particles.copy()
        self.personal_scores = np.array([objective_function(p) for p in self.particles])
        self.global_best = self.particles[np.argmin(self.personal_scores)]
        self.global_score = np.min(self.personal_scores)
        self.final_iteration = 0
        self.history = []
        
    def update(self, frame):
        self.final_iteration = frame + 1
        r1 = np.random.rand(NUM_PARTICLES, DIMENSIONS)
        r2 = np.random.rand(NUM_PARTICLES, DIMENSIONS)
        
        cognitive = COGNITIVE_WEIGHT * r1 * (self.personal_bests - self.particles)
        social = SOCIAL_WEIGHT * r2 * (self.global_best - self.particles)
        
        self.velocities = (INERTIA_WEIGHT * self.velocities + cognitive + social)
        self.particles += self.velocities
        
        self.particles = np.clip(self.particles, SEARCH_SPACE[0], SEARCH_SPACE[1])
        
        current_scores = np.array([objective_function(p) for p in self.particles])
        improved = current_scores < self.personal_scores
        self.personal_bests[improved] = self.particles[improved]
        self.personal_scores[improved] = current_scores[improved]
        
        if np.min(current_scores) < self.global_score:
            self.global_best = self.particles[np.argmin(current_scores)]
            self.global_score = np.min(current_scores)
        
        particles_scatter.set_offsets(self.particles)
        global_best_scatter.set_offsets([self.global_best])
        
        self.history.append({
            'particles': self.particles.copy(),
            'global_best': self.global_best.copy(),
            'score': self.global_score
        })
        
        ax.set_title(f'PSO Iteration {frame+1}\nBest Score: {self.global_score:.4f}')
        
        if self.global_score < 0.001:
            ani.event_source.stop()
        
        return particles_scatter, global_best_scatter
    
    def get_final_results(self):
        return {
            "best_position": self.global_best,
            "best_score": self.global_score,
            "iterations": self.final_iteration,
            "particles_history": self.history
        }

# Create animation
pso = PSO_Visualizer()
ani = FuncAnimation(fig, pso.update, frames=NUM_ITERATIONS, interval=200, blit=True)

# Save as GIF
gif_path = "pso_animation.gif"
writer = PillowWriter(fps=10)
ani.save(gif_path, writer=writer)

# Print final results
final_results = pso.get_final_results()
print("\n=== FINAL RESULTS ===")
print(f"Best Position: {final_results['best_position']}")
print(f"Best Score (MSE): {final_results['best_score']:.6f}")
print(f"Total Iterations: {final_results['iterations']}")
print(f"Target Position: {TARGET_ARRAY}")

# Additional analysis
distance_to_target = np.linalg.norm(final_results['best_position'] - TARGET_ARRAY)
print(f"\nDistance to Target: {distance_to_target:.6f}")

if final_results['iterations'] < NUM_ITERATIONS:
    print("PSO converged early!")
else:
    print("PSO completed all iterations.")

plt.close()


=== FINAL RESULTS ===
Best Position: [ 0.99794249 -0.99970206]
Best Score (MSE): 0.000000
Total Iterations: 100
Target Position: [ 1 -1]

Distance to Target: 0.002079
PSO completed all iterations.
